# Menjalankan *python* dan *mongodb* di *Google Colab*

Disusun oleh:
*   Pak Ruhul 
*   Pak Sopian
*   Pak Triawan

# 1. Instalasi mongodb

In [2]:
# install pymongo
!python -m pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# install mongodb
!apt install mongodb >log
!service mongodb start



 * Starting database mongodb
   ...done.


In [ ]:
!mongo

MongoDB shell version v3.6.3
connecting to: mongodb://127.0.0.1:27017
MongoDB server version: 3.6.3
Server has startup warnings: 
2022-07-31T02:48:26.218+0000 I CONTROL  [initandlisten] 
2022-07-31T02:48:26.218+0000 I CONTROL  [initandlisten] ** WARNING: Access control is not enabled for the database.
2022-07-31T02:48:26.218+0000 I CONTROL  [initandlisten] **          Read and write access to data and configuration is unrestricted.
2022-07-31T02:48:26.218+0000 I CONTROL  [initandlisten] 
> show dbsshow dbs
admin        0.000GB
config       0.000GB
db_akademik  0.000GB
local        0.000GB
> use db_akademikuse db_akademik
switched to db db_akademik
> db.dropDatabase()db.dropDatabase()
{ "dropped" : "db_akademik", "ok" : 1 }
> 

# 2. Koneksi ke mongodb dengan python

In [4]:
# cek list database, untuk mengetahui apakah mongodb sudah berhasil terinstall
from pymongo import MongoClient
client = MongoClient()
if client:
  print("python sudah terhubung dengan mongodb")
else:
  print("gagal terhubung ke mongodb")

python sudah terhubung dengan mongodb


In [5]:
# cek detail koneksi client
client.stat

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'stat')

## 2.1 Membuat database, collection dan insert satu data di mongodb dengan python

In [6]:
from pymongo import MongoClient
mongo_url = "mongodb://localhost:27017/"
mongo_db = MongoClient(mongo_url)

#create db
db = mongo_db["db_akademik"]

#create collection
coll = db.create_collection("mahasiswa")

#create document
data = dict()

data["nim"] = 5184025
data["nama"] = "ruhul"

x = coll.insert_one(data).inserted_id

## 2.2 Insert multi data di mongodb dengan python

In [9]:
#create document
data = [
    {"nim": 5183027, "nama": "Sopian Alviana"},
    {"nim": 5183034, "nama": "Defiana Arnaldy"},
    {"nim": 5183046, "nama": "Agus Mulyana"},
]

x = coll.insert_many(data)
print(x.inserted_ids)

[ObjectId('62e62305e128410f019edf33'), ObjectId('62e62305e128410f019edf34'), ObjectId('62e62305e128410f019edf35')]


## 2.3 Menampilkan isi data collection "mahasiswa"

In [8]:
from pymongo import MongoClient

if mongo_url:
    db = mongo_db["db_akademik"]
    coll = db.get_collection("mahasiswa")
    isidata = list(coll.find())
    for data in isidata:
        print(" nim: {},\n nama: {}".format(data["nim"], data["nama"]))
else:
    print("kode program tidak terhubung ke database")

 nim: 5184025,
 nama: ruhul


## 2.4 Menampilkan isi data collection "mahasiswa" dengan kondisi tertentu

In [11]:
query = coll.find({"nim": {"$lt": 5184025}})

print("Data pelajar yang memiliki nim dibawah 5184025 adalah: ")
for data in query:
    print(data)

Data pelajar yang memiliki nim dibawah 5184025 adalah: 
{'_id': ObjectId('62e62305e128410f019edf33'), 'nim': 5183027, 'nama': 'Sopian Alviana'}
{'_id': ObjectId('62e62305e128410f019edf34'), 'nim': 5183034, 'nama': 'Defiana Arnaldy'}
{'_id': ObjectId('62e62305e128410f019edf35'), 'nim': 5183046, 'nama': 'Agus Mulyana'}


## 2.5 Menampilkan isi data collection "mahasiswa" dengan kondisi terurut berdasarkan atribut khusus, misal dari urutan huruf awal->akhir, angka terkecil->terbesar, dan lain-lain

In [12]:
query = coll.find().sort("nama")

print("Urutan nama pelajar berdasarkan awalan nama adalah: ")
for data in query:
    print(data)

Urutan nama pelajar berdasarkan awalan nama adalah: 
{'_id': ObjectId('62e62305e128410f019edf35'), 'nim': 5183046, 'nama': 'Agus Mulyana'}
{'_id': ObjectId('62e62305e128410f019edf34'), 'nim': 5183034, 'nama': 'Defiana Arnaldy'}
{'_id': ObjectId('62e62305e128410f019edf33'), 'nim': 5183027, 'nama': 'Sopian Alviana'}
{'_id': ObjectId('62e61ea7e128410f019edf32'), 'nim': 5184025, 'nama': 'ruhul'}


## 2.6 Memperbaharui satu data di collection "mahasiswa"

In [13]:
query = { "nim": 5184025}
value = { "$set": { "nama": "Ruhul Amin" } }

coll.update_one(query, value)

#tampilkan nama "mahasiswa" yang sudah terupdate
for x in coll.find():
  print(x)

{'_id': ObjectId('62e61ea7e128410f019edf32'), 'nim': 5184025, 'nama': 'Ruhul Amin'}
{'_id': ObjectId('62e62305e128410f019edf33'), 'nim': 5183027, 'nama': 'Sopian Alviana'}
{'_id': ObjectId('62e62305e128410f019edf34'), 'nim': 5183034, 'nama': 'Defiana Arnaldy'}
{'_id': ObjectId('62e62305e128410f019edf35'), 'nim': 5183046, 'nama': 'Agus Mulyana'}


## 2.7 Memperbaharui multi data di collection "mahasiswa"

In [18]:
query = { "nim": {"$gt": 5183034}}
value = { "$set": { "promotor": "Prof. Imas Sukaesih Sitanggang"} }

x = coll.update_many(query, value)

#cek jumlah dokumen yang berhasil diperbaharui
print(x.modified_count, "Dokumen sudah diperbaharui.")

#tampilkan semua data dengan nim kurang dari 5183034 dan 
#menambahkan atribut "promotor" pada data tersebut
for x in coll.find():
  print(x)

2 Dokumen sudah diperbaharui.
{'_id': ObjectId('62e61ea7e128410f019edf32'), 'nim': 5184025, 'nama': 'Ruhul Amin', 'promotor': 'Prof. Imas Sukaesih Sitanggang'}
{'_id': ObjectId('62e62305e128410f019edf33'), 'nim': 5183027, 'nama': 'Sopian Alviana', 'promotor': 'Prof. Taufik Djatna'}
{'_id': ObjectId('62e62305e128410f019edf34'), 'nim': 5183034, 'nama': 'Defiana Arnaldy'}
{'_id': ObjectId('62e62305e128410f019edf35'), 'nim': 5183046, 'nama': 'Agus Mulyana', 'promotor': 'Prof. Imas Sukaesih Sitanggang'}


## 2.8 Menghapus satu data pada collection "mahasiswa"

In [19]:
query = {"nim": 5183034}
coll.delete_one(query)

for data in coll.find():
    print(data)

{'_id': ObjectId('62e61ea7e128410f019edf32'), 'nim': 5184025, 'nama': 'Ruhul Amin', 'promotor': 'Prof. Imas Sukaesih Sitanggang'}
{'_id': ObjectId('62e62305e128410f019edf33'), 'nim': 5183027, 'nama': 'Sopian Alviana', 'promotor': 'Prof. Taufik Djatna'}
{'_id': ObjectId('62e62305e128410f019edf35'), 'nim': 5183046, 'nama': 'Agus Mulyana', 'promotor': 'Prof. Imas Sukaesih Sitanggang'}


## 2.9 Menghapus multi data pada collection "mahasiswa"

In [20]:
query = {"nim": {"$gt": 5183034}}

x = coll.delete_many(query)

print(x.deleted_count, " dokumen terhapus.")
for x in coll.find():
  print(x)

2  dokumen terhapus.
{'_id': ObjectId('62e62305e128410f019edf33'), 'nim': 5183027, 'nama': 'Sopian Alviana', 'promotor': 'Prof. Taufik Djatna'}


# 3. Operasi dasar manajemen database mongodb

In [ ]:
# perintah untuk menampilkan database yang sudah ada
client.list_database_names()

['admin', 'config', 'local']

In [ ]:
# perintah untuk menggunakan database tertentu, apabila belum ada database dengan
# nama tersebut maka perintah ini akan mengeksekusi untuk membuat database dengan
# nama sesuai masukkan
db = client.db_akademik
client.list_database_names()

['admin', 'config', 'local']

In [ ]:
# perintah untuk menghapus database
client.drop_database('db_akademik')
client.list_database_names()

['admin', 'config', 'local']

# 4. Operasi dasar manajemen collection

In [ ]:
# perintah untuk membuat collection
db.create_collection('mahasiswa')
db.create_collection('matakuliah')
db.create_collection('pengambilanmk')
list(db.list_collections())

[{'idIndex': {'key': {'_id': 1},
   'name': '_id_',
   'ns': 'db_akademik.pengambilanmk',
   'v': 2},
  'info': {'readOnly': False,
   'uuid': Binary(b'\x16\xa93\x83\x16\xe6L\xe2\xba\x88\x10\x93\xdd\xb8\x12c', 4)},
  'name': 'pengambilanmk',
  'options': {},
  'type': 'collection'},
 {'idIndex': {'key': {'_id': 1},
   'name': '_id_',
   'ns': 'db_akademik.mahasiswa',
   'v': 2},
  'info': {'readOnly': False,
   'uuid': Binary(b'3H}\xff\x80/E\xb5\x83\x0ej\x8emIn*', 4)},
  'name': 'mahasiswa',
  'options': {},
  'type': 'collection'},
 {'idIndex': {'key': {'_id': 1},
   'name': '_id_',
   'ns': 'db_akademik.mhs',
   'v': 2},
  'info': {'readOnly': False,
   'uuid': Binary(b'\x9a9\x13\xc9E\x1dB4\x98\xfdJbO\x17\xb1\x1b', 4)},
  'name': 'mhs',
  'options': {},
  'type': 'collection'},
 {'idIndex': {'key': {'_id': 1},
   'name': '_id_',
   'ns': 'db_akademik.matakuliah',
   'v': 2},
  'info': {'readOnly': False,
   'uuid': Binary(b'q\xd5hg\xe6DA\xd8\x92u}\x12\x91\x0b\xd9G', 4)},
  'name': 'm

In [ ]:
# perintah untuk memasukkan satu data ke collection mahasiswa
coll = db.mahasiswa
coll.insert_one({"nim": 5184025, "nama": "Ruhul Amin"})
list(coll.find())

[{'_id': ObjectId('62e2c6113c002fc09f650884'),
  'nama': 'Ruhul Amin',
  'nim': 5184025}]

In [ ]:
# perintah untuk memasukkan lebih dari satu data ke collection
coll = db.mahasiswa
coll.insert_many([{"nim": 5183027, "nama": "Sopian"}, # kalau lebih dari satu record wajib berbentuk list
                  {"nim": 5184088, "nama": "Triawan"}, 
                  {"nim": 5184097, "nama": "Sapto Budi "}
                  ])
list(coll.find())

[{'_id': ObjectId('62e2c6113c002fc09f650884'),
  'nama': 'Ruhul Amin',
  'nim': 5184025},
 {'_id': ObjectId('62e2c61c3c002fc09f650885'),
  'nama': 'Sopian',
  'nim': 5183027},
 {'_id': ObjectId('62e2c61c3c002fc09f650886'),
  'nama': 'Triawan',
  'nim': 5184088},
 {'_id': ObjectId('62e2c61c3c002fc09f650887'),
  'nama': 'Sapto Budi ',
  'nim': 5184097}]

In [ ]:
# mengubah nama collection
db.mahasiswa.rename('mhs')
coll = db.mhs                           # Set the collection to work with
list(db.list_collections())                  # Show the existing collections

[{'idIndex': {'key': {'_id': 1},
   'name': '_id_',
   'ns': 'db_akademik.pengambilanmk',
   'v': 2},
  'info': {'readOnly': False,
   'uuid': Binary(b'\x16\xa93\x83\x16\xe6L\xe2\xba\x88\x10\x93\xdd\xb8\x12c', 4)},
  'name': 'pengambilanmk',
  'options': {},
  'type': 'collection'},
 {'idIndex': {'key': {'_id': 1},
   'name': '_id_',
   'ns': 'db_akademik.mhs',
   'v': 2},
  'info': {'readOnly': False,
   'uuid': Binary(b'3H}\xff\x80/E\xb5\x83\x0ej\x8emIn*', 4)},
  'name': 'mhs',
  'options': {},
  'type': 'collection'},
 {'idIndex': {'key': {'_id': 1},
   'name': '_id_',
   'ns': 'db_akademik.matakuliah',
   'v': 2},
  'info': {'readOnly': False,
   'uuid': Binary(b'q\xd5hg\xe6DA\xd8\x92u}\x12\x91\x0b\xd9G', 4)},
  'name': 'matakuliah',
  'options': {},
  'type': 'collection'}]

In [ ]:
# hapus collection
db.drop_collection('pengambilanmk')

{'nIndexesWas': 1, 'ns': 'db_akademik.pengambilanmk', 'ok': 1.0}

# 5. Operasi dasar collection

## 5.1 Operasi Create

In [ ]:
# membuat dokumen dengan insert_one dan javascript notation
#@title Dokumen Mahasiswa

Nim = 5184040 #@param {type: "integer"}
Nama = "Fahmi" #@param {type:"string"}


In [ ]:
# memasukkan atribut/variabel tersebut ke collection
data = {"nim": Nim,
        "nama": Nama}
insertdata = coll.insert_one(data)

In [ ]:
# konfirmasi apakah berhasil 
insertdata.acknowledged

True

In [ ]:
# menampilkan id dokumen
insertdata.inserted_id

ObjectId('62e2c65e3c002fc09f650888')

## 5.2 Operasi Read

In [ ]:
# menampilkan/membaca isi collection
list(coll.find())

[{'_id': ObjectId('62e2c6113c002fc09f650884'),
  'nama': 'Ruhul Amin',
  'nim': 5184025},
 {'_id': ObjectId('62e2c61c3c002fc09f650885'),
  'nama': 'Sopian',
  'nim': 5183027},
 {'_id': ObjectId('62e2c61c3c002fc09f650886'),
  'nama': 'Triawan',
  'nim': 5184088},
 {'_id': ObjectId('62e2c61c3c002fc09f650887'),
  'nama': 'Sapto Budi ',
  'nim': 5184097},
 {'_id': ObjectId('62e2c65e3c002fc09f650888'),
  'nama': 'Fahmi ',
  'nim': 5184040}]

In [ ]:
# menampilkan data berdasarkan dokumen id tertentu
list(coll.find({'_id': insertdata.inserted_id }))

[{'_id': ObjectId('62e2c65e3c002fc09f650888'),
  'nama': 'Fahmi ',
  'nim': 5184040}]

In [ ]:
# menampilkan data berdasarkan atribut/field "Nim"
list(coll.find({'nim': Nim}))

[{'_id': ObjectId('62e2c65e3c002fc09f650888'),
  'nama': 'Fahmi ',
  'nim': 5184040}]

In [ ]:
# menampilkan data berdasarkan atribut/field "Nama"
list(coll.find({'nama': Nama}))

[{'_id': ObjectId('62e2c65e3c002fc09f650888'),
  'nama': 'Fahmi ',
  'nim': 5184040}]

In [ ]:
# menampilkan data dengan jumlah yang dibatasi sesuai keinginan, misal: 2
list(coll.find().limit(2)) 

[{'_id': ObjectId('62e2c6113c002fc09f650884'),
  'nama': 'Ruhul Amin',
  'nim': 5184025},
 {'_id': ObjectId('62e2c61c3c002fc09f650885'),
  'nama': 'Sopian',
  'nim': 5183027}]

In [ ]:
# menampilkan data dengan melewati data pada posisi tertentu, misal: posisi ke 1 (indeks ke 0) dilewati
list(coll.find().skip(1)) # tidak ada nama "ruhul amin" yang tampil

[{'_id': ObjectId('62e2c61c3c002fc09f650885'),
  'nama': 'Sopian',
  'nim': 5183027},
 {'_id': ObjectId('62e2c61c3c002fc09f650886'),
  'nama': 'Triawan',
  'nim': 5184088},
 {'_id': ObjectId('62e2c61c3c002fc09f650887'),
  'nama': 'Sapto Budi ',
  'nim': 5184097},
 {'_id': ObjectId('62e2c65e3c002fc09f650888'),
  'nama': 'Fahmi ',
  'nim': 5184040}]

## 5.3 Operasi Update

In [ ]:
## update data yang sudah ada di collection
updatedata = coll.update_one( 
    {'nama': Nama}, 
    {'$set': { 'usia': 30 }} 
    ) 

list(coll.find({'nama': Nama }))

[{'_id': ObjectId('62e2c65e3c002fc09f650888'),
  'nama': 'Fahmi ',
  'nim': 5184040,
  'usia': 30}]

In [ ]:
# selain menggunakan perintah diatas, update data bisa juga menggunakan perintah seperti berikut ini
coll.find_one_and_update({'nama': Nama}, {'$set': {'usia': 31}})

{'_id': ObjectId('62e2c65e3c002fc09f650888'),
 'nama': 'Fahmi ',
 'nim': 5184040,
 'usia': 31}

In [ ]:
# cek apakah data berhasil berubah/update atau tidak
updatedata.raw_result

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

## 5.4 Operasi Delete

In [21]:
coll.delete_one({"nama": "Fahmi"})
list(coll.find())

[{'_id': ObjectId('62e62305e128410f019edf33'),
  'nama': 'Sopian Alviana',
  'nim': 5183027,
  'promotor': 'Prof. Taufik Djatna'}]